In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train=pd.read_parquet('/content/drive/MyDrive/NK-securities/train_data.parquet')


In [ ]:
train_df=pd.read_parquet('/content/drive/MyDrive/NK-securities/train_data.parquet')


In [ ]:
test_df=pd.read_parquet('/content/drive/MyDrive/NK-securities/test_data.parquet')

In [ ]:
test=pd.read_parquet('/content/drive/MyDrive/NK-securities/test_data.parquet')

In [ ]:
sample_submission_df=pd.read_csv('/content/drive/MyDrive/NK-securities/sample_submission.csv')

In [ ]:
test_df=pd.read_parquet('/content/test_data.parquet')

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Normalize features (including 'underlying')
feature_cols = [f'X{i}' for i in range(42) if f'X{i}' in test_df.columns] + ['underlying']
scaler = StandardScaler()
test_df[feature_cols] = scaler.fit_transform(test_df[feature_cols])


In [ ]:

numeric_cols = test_df.select_dtypes(include='number').columns



In [ ]:
columns_to_drop = ['underlying'] + [f'X{i}' for i in range(42)]
numeric_cols = numeric_cols.drop(columns_to_drop, errors='ignore')

In [ ]:
numeric_cols.drop('timestamp')

Index(['call_iv_24000', 'call_iv_24100', 'call_iv_24200', 'call_iv_24300',
       'call_iv_24400', 'call_iv_24500', 'call_iv_24600', 'call_iv_24700',
       'call_iv_24800', 'call_iv_24900', 'call_iv_25000', 'call_iv_25100',
       'call_iv_25200', 'call_iv_25300', 'call_iv_25400', 'call_iv_25500',
       'call_iv_25600', 'call_iv_25700', 'call_iv_25800', 'call_iv_25900',
       'call_iv_26000', 'call_iv_26100', 'call_iv_26200', 'call_iv_26300',
       'call_iv_26400', 'call_iv_26500', 'put_iv_23000', 'put_iv_23100',
       'put_iv_23200', 'put_iv_23300', 'put_iv_23400', 'put_iv_23500',
       'put_iv_23600', 'put_iv_23700', 'put_iv_23800', 'put_iv_23900',
       'put_iv_24000', 'put_iv_24100', 'put_iv_24200', 'put_iv_24300',
       'put_iv_24400', 'put_iv_24500', 'put_iv_24600', 'put_iv_24700',
       'put_iv_24800', 'put_iv_24900', 'put_iv_25000', 'put_iv_25100',
       'put_iv_25200', 'put_iv_25300', 'put_iv_25400', 'put_iv_25500'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Alternative: Simplified version with pre-tuned parameters
def fast_iterative_rf_imputation(test_df_path, output_path='test_fast_rf_imputed.csv'):
    """
    Faster version with pre-optimized Random Forest parameters
    """
    test_df = pd.read_parquet(test_df_path)

    # Identify columns
    feature_cols = [f'X{i}' for i in range(42) if f'X{i}' in test_df.columns] + ['underlying']
    iv_cols = [col for col in test_df.columns if 'iv_' in col]

    # Normalize features
    scaler = StandardScaler()
    test_df[feature_cols] = scaler.fit_transform(test_df[feature_cols])

    # Optimized Random Forest parameters for volatility data
    rf_imputer = IterativeImputer(
        estimator=RandomForestRegressor(
            n_estimators=150,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features='sqrt',
            bootstrap=True,
            random_state=42,
            n_jobs=-1
        ),
        max_iter=20,
        tol=1e-4,
        initial_strategy='median',
        imputation_order='ascending',
        random_state=42,
        verbose=1
    )

    # Store original missing pattern
    original_missing = test_df[iv_cols].isna()

    # Impute
    print("Performing Random Forest imputation...")
    imputed_data = rf_imputer.fit_transform(test_df[iv_cols])

    # Create final result
    final_df = test_df.copy()
    imputed_df = pd.DataFrame(imputed_data, columns=iv_cols, index=test_df.index)

    # Fill only originally missing values
    for col in iv_cols:
        missing_mask = original_missing[col]
        final_df.loc[missing_mask, col] = imputed_df.loc[missing_mask, col]

    # Clip and clean
    final_df[iv_cols] = final_df[iv_cols].clip(0.01, 2.0).fillna(0.2)

    # Save
    final_df.to_csv(output_path, index=False)
    print(f"Fast Random Forest imputation complete! Saved to: {output_path}")

    return final_df


In [ ]:
# For faster execution with good defaults:
result_df = fast_iterative_rf_imputation('/content/test_data.parquet')

Performing Random Forest imputation...
[IterativeImputer] Completing matrix with shape (12065, 52)
[IterativeImputer] Change: 2.583419987176378, scaled tolerance: 5.367150000000001e-05 
[IterativeImputer] Change: 0.43220699015041053, scaled tolerance: 5.367150000000001e-05 
[IterativeImputer] Change: 0.24479971503235978, scaled tolerance: 5.367150000000001e-05 
[IterativeImputer] Change: 0.09894365602862248, scaled tolerance: 5.367150000000001e-05 
[IterativeImputer] Change: 0.06737499006258607, scaled tolerance: 5.367150000000001e-05 
[IterativeImputer] Change: 0.05313382299953853, scaled tolerance: 5.367150000000001e-05 
[IterativeImputer] Change: 0.03636763733936321, scaled tolerance: 5.367150000000001e-05 
[IterativeImputer] Change: 0.03275710864428255, scaled tolerance: 5.367150000000001e-05 
[IterativeImputer] Change: 0.02954596079084232, scaled tolerance: 5.367150000000001e-05 
[IterativeImputer] Change: 0.0208989722769706, scaled tolerance: 5.367150000000001e-05 
[IterativeImpu

In [ ]:
# # Usage
# #test_df = pd.read_csv('test_subset_500.csv')
# imputed_df = xgb_mice_imputation(test_df)
# imputed_df.to_csv('test_xgb_mice_imputed.csv', index=False)

<ipython-input-14-b67a056c3092>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_imputed[col].fillna(test_imputed[col].median(), inplace=True)


Iteration 1/10
  Mean change: 0.017308
Iteration 2/10
  Mean change: 0.000528
Iteration 3/10
  Mean change: 0.000318
Iteration 4/10
  Mean change: 0.000292
Iteration 5/10
  Mean change: 0.000290
Iteration 6/10
  Mean change: 0.000274
Iteration 7/10
  Mean change: 0.000269
Iteration 8/10
  Mean change: 0.000274
Iteration 9/10
  Mean change: 0.000280
Iteration 10/10
  Mean change: 0.000281


In [ ]:
# imputed_df

,timestamp,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,call_iv_24800,...,put_iv_24600,put_iv_24700,put_iv_24800,put_iv_24900,put_iv_25000,put_iv_25100,put_iv_25200,put_iv_25300,put_iv_25400,put_iv_25500
0,0,0.280939,0.259713,0.257516,0.251349,0.242149,0.238442,0.232439,0.225217,0.222997,...,0.232334,0.225419,0.221302,0.227301,0.234169,0.244913,0.250422,0.258065,0.274296,0.282229
1,1,0.270276,0.270495,0.258893,0.250655,0.244487,0.239770,0.233548,0.228426,0.224157,...,0.234077,0.228209,0.223889,0.229982,0.239068,0.249402,0.257737,0.264442,0.271163,0.283825
2,2,0.257354,0.251731,0.235410,0.224285,0.214869,0.204580,0.194604,0.188580,0.183306,...,0.194612,0.188052,0.183136,0.180902,0.181346,0.184670,0.190750,0.196791,0.203418,0.211896
3,3,0.241888,0.230321,0.220505,0.208274,0.198602,0.186190,0.176561,0.165584,0.162550,...,0.175959,0.166394,0.161561,0.159621,0.163064,0.172032,0.180905,0.189175,0.196888,0.206107
4,4,0.235328,0.230220,0.222983,0.214126,0.205910,0.198878,0.192603,0.186194,0.181544,...,0.192590,0.185940,0.181446,0.177920,0.175159,0.176000,0.177520,0.179145,0.182314,0.186674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12060,12060,0.241911,0.233427,0.226570,0.216564,0.206694,0.199832,0.192848,0.186856,0.181464,...,0.192783,0.186228,0.181381,0.177270,0.175185,0.174460,0.176356,0.177716,0.179217,0.182839
12061,12061,0.245131,0.236380,0.226972,0.216749,0.208343,0.200906,0.193655,0.187681,0.183216,...,0.194018,0.187526,0.182981,0.180196,0.177969,0.177923,0.179091,0.180254,0.181427,0.183981
12062,12062,0.256475,0.243817,0.234267,0.221270,0.210935,0.201559,0.192119,0.185088,0.179760,...,0.192306,0.185780,0.179666,0.176916,0.176743,0.181503,0.187089,0.193951,0.201360,0.209940
12063,12063,0.247958,0.239825,0.231997,0.222515,0.212660,0.205103,0.196607,0.190833,0.185851,...,0.197198,0.190933,0.186172,0.181329,0.178694,0.177644,0.177963,0.178806,0.180236,0.181946


In [ ]:
# prompt: remove columns underlying and X0 to X41 from final result

# Remove 'underlying' and 'X' columns
columns_to_drop = ['underlying'] + [f'X{i}' for i in range(42)]
result_df = result_df.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
result_df

,timestamp,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,call_iv_24800,...,put_iv_24600,put_iv_24700,put_iv_24800,put_iv_24900,put_iv_25000,put_iv_25100,put_iv_25200,put_iv_25300,put_iv_25400,put_iv_25500
0,0,0.280939,0.265874,0.257090,0.249550,0.242149,0.237232,0.232439,0.225789,0.222997,...,0.232334,0.226011,0.222287,0.227301,0.234169,0.244410,0.250422,0.258174,0.274101,0.282229
1,1,0.270276,0.267651,0.258893,0.250757,0.244349,0.239160,0.233548,0.228039,0.225082,...,0.234093,0.228209,0.224904,0.229991,0.239069,0.249402,0.258161,0.264399,0.271855,0.285570
2,2,0.256486,0.251731,0.236903,0.224577,0.214869,0.204580,0.194604,0.188285,0.183418,...,0.194612,0.188052,0.183138,0.180765,0.181346,0.185262,0.190750,0.196924,0.204615,0.212302
3,3,0.241888,0.232152,0.220505,0.208652,0.198602,0.186190,0.174879,0.167021,0.162071,...,0.174207,0.166394,0.161561,0.160825,0.165028,0.172032,0.181296,0.189826,0.197537,0.206107
4,4,0.235328,0.229726,0.222983,0.214126,0.206162,0.199196,0.192603,0.186249,0.181497,...,0.192682,0.186068,0.181443,0.177920,0.175760,0.176000,0.177706,0.178931,0.182314,0.186338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12060,12060,0.242214,0.233427,0.225918,0.215985,0.207396,0.200011,0.192748,0.186856,0.181783,...,0.192982,0.186414,0.181734,0.177270,0.174958,0.174779,0.176356,0.177716,0.179500,0.182586
12061,12061,0.243454,0.236380,0.226972,0.217002,0.208548,0.200906,0.193655,0.187680,0.183725,...,0.193764,0.187554,0.183553,0.180196,0.177969,0.177923,0.179070,0.180254,0.181813,0.184073
12062,12062,0.256475,0.243817,0.234267,0.221270,0.211462,0.201356,0.192140,0.185088,0.179760,...,0.192306,0.184819,0.179666,0.176916,0.177407,0.181607,0.187479,0.194130,0.201360,0.209940
12063,12063,0.249071,0.239825,0.231117,0.222167,0.212967,0.204975,0.197394,0.190833,0.185851,...,0.197758,0.190933,0.186172,0.181858,0.178350,0.177548,0.177790,0.178806,0.180040,0.182376


In [ ]:
# # prompt: download imputed_df as csv now

# from google.colab import files

# result_df.to_csv('imputed_dataframe.csv', index=False)
# files.download('imputed_dataframe.csv')
# print("Imputed DataFrame downloaded as imputed_dataframe.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Imputed DataFrame downloaded as imputed_dataframe.csv


In [ ]:
# vamshi=pd.read_csv('/content/vamshi_submission.csv')

In [ ]:
# # prompt: find MSE between vamshi and imputed_df

# from sklearn.metrics import mean_squared_error
# # Align the columns between the two DataFrames before calculating MSE
# common_cols = list(set(vamshi.columns) & set(imputed_df.columns))
# vamshi_aligned = vamshi[common_cols]
# imputed_df_aligned = imputed_df[common_cols]
# mse = mean_squared_error(vamshi_aligned, imputed_df_aligned)
# print(f"Mean Squared Error between vamshi and imputed_df: {mse}")

Mean Squared Error between vamshi and imputed_df: 8.314045176927046e-06
